In [83]:
import sys
sys.path.append('../..')

import csv
import codecs
import copy
import re
from random import shuffle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB

import nltk
import spacy
import gensim
from gensim.models.phrases import Phrases, Phraser
from nltk import word_tokenize
from nltk.corpus import stopwords

from utils import tokenizer, load_six_emotions, load_3_emotions

### Dataset

Para validar, serão utilizados dois datasets.

O primeiro deles foi fornecido a nós pela Barbara Martinazzo, e contém cerca de 1000 frases categorizadas entre as 6 emoções do Ekman (alegria, surpresa, tristeza, medo, raiva, desgosto).

O segundo dataset, contém também cerca de 1000 frases, também categorizados usando as 6 emoções conforme o anterior.

In [84]:
def carregar(filename):
    frases = []
    with open(filename, 'r') as h:
        reader = csv.reader(h, delimiter=';')
        for row in reader:
            frase = tokenizer(row[0]).strip()
            sentimento = row[1].upper()
            if len(frase) > 5:
                frases.append((sentimento, frase))
    return frases

frases = carregar('/home/rdenadai/vagrant/python-dev/sentiment-analysis-2018-president-election/dataset/puc-pr/politica.csv')
frases += carregar('/home/rdenadai/vagrant/python-dev/sentiment-analysis-2018-president-election/dataset/puc-pr/g1_v1.csv')

shuffle(frases)

print(frases[:5])

[('DESGOSTO', 'turqu descart terror ataqu casament tir casament deix mort ministr diss ataqu motiv brig famíl'), ('DESGOSTO', 'atent paquistã mat oit pesso bomb explod interior um carr provínc fronteir noro outr pesso fic fer'), ('ALEGRIA', 'risc cânc cai metad minut exercíci dia diz estud univers finlandes avali homens anos idad trabalh public british journal of sports medicin'), ('TRISTEZA', 'motocicl morr após ser ating linh cerol mg rapaz anos pass br govern valad segund políc não habilit dirig mot'), ('DESGOSTO', 'nov grip contamin pesso país segund oms estad unid país maior númer cas agênc onu diz pior pod aind não ter pass')]


In [86]:
afrases = []
asentimentos =[]
for sentimento, frase in frases:
    afrases.append(frase)
    asentimentos.append(sentimento)

In [87]:
vec_tfidf = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf = vec_tfidf.fit_transform(afrases)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, asentimentos, test_size=0.3, random_state=42)

In [88]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.5714285714285714

In [89]:
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
X_svd = svd.fit_transform(X_tfidf)
X_train, X_test, y_train, y_test = train_test_split(X_svd, asentimentos, test_size=0.3, random_state=42)

In [90]:
clf = GaussianNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.4568106312292359

In [91]:
lda = LatentDirichletAllocation(n_components=5, random_state=0)
X_lda = lda.fit_transform(X_tfidf)
X_train, X_test, y_train, y_test = train_test_split(X_lda, asentimentos, test_size=0.3, random_state=42)

In [92]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.3754152823920266